In [5]:
import os

def kill_msedge_processes():
    # /F: force kill
    # /IM: image name (process name)
    os.system("taskkill /F /IM msedge.exe")

kill_msedge_processes()

In [6]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

In [7]:
def sync_edge_account(driver):
    pass
#     """
#     Click vào nút "Sign in to sync data" để đồng bộ tài khoản Edge
    
#     Args:
#         driver: WebDriver instance
    
#     Returns:
#         bool: True nếu click thành công, False nếu không tìm thấy
#     """
#     try:        
#         # Các selector CSS hợp lệ cho nút sync (loại bỏ :contains())
#         sync_selectors = [
#             "button[aria-label*='Sign in']",  # Nút có chữ Sign in trong aria-label
#             "button[title*='Sign in']",
#             "button[data-test-id*='sync']",
#             "a[href*='login']",
#             "[data-automation-id*='signin']",
#             ".profile-signin-button",
#             "#signin-button"
#         ]
        
#         for selector in sync_selectors:
#             try:
#                 sync_button = WebDriverWait(driver, 1).until(
#                     EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
#                 )
                
#                 if sync_button:
#                     print(f"✅ Tìm thấy nút sync: {sync_button.text}")
#                     sync_button.click()
#                     print("👆 Đã click vào nút sync!")
#                     time.sleep(2)  # Đợi sync hoàn tất
#                     return True
                    
#             except (TimeoutException, NoSuchElementException):
#                 continue
        
#         # Thử tìm bằng XPath dựa trên text chứa 'Sign in' hoặc 'sync'
#         try:
#             sync_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'Sign in') or contains(text(), 'sync')]")
#             for element in sync_elements:
#                 if element.is_displayed() and element.is_enabled():
#                     print(f"✅ Tìm thấy element sync: {element.text}")
#                     element.click()
#                     print("👆 Đã click vào element sync!")
#                     time.sleep(1)
#                     return True
#         except Exception:
#             pass
            
#         print("ℹ️  Không tìm thấy nút 'Sign in to sync data' - có thể đã đăng nhập rồi")
#         return False
        
#     except Exception as e:
#         print(f"⚠️  Lỗi khi tìm nút sync: {e}")
#         return False

In [8]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.edge.service import Service
import os

def auto_bing_search(keywords, profile_path=None, profile_name="Default", use_sync=False):
    kill_msedge_processes()
    """
    Tự động tìm kiếm trên Bing và click vào kết quả đầu tiên
    
    Args:
        keywords (list): Danh sách từ khóa cần tìm kiếm
        profile_path (str): Đường dẫn đầy đủ đến thư mục User Data của Edge
        profile_name (str): Tên profile cần sử dụng (Default, Profile 1, Profile 2, ...)
        use_sync (bool): Có click nút sync không
    """
    for i, keyword in enumerate(keywords, 1):
        print(f"/n{'='*60}")
        print(f"🔍 [{i}/{len(keywords)}] Đang xử lý từ khóa: '{keyword}'")
        print(f"{'='*60}")
        
        # Khởi tạo Edge browser mới cho mỗi từ khóa với profile cố định
        edge_options = Options()
        
        # Thiết lập profile với đường dẫn đầy đủ
        edge_options.add_argument(f"--user-data-dir={profile_path}")
        edge_options.add_argument(f"--profile-directory={profile_name}")
        
        # Các options khác
        edge_options.add_argument("--window-size=1920,1080")
        edge_options.add_argument("--edge-skip-compat-layer-relaunch")
        edge_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0")
        edge_options.add_argument("--disable-blink-features=AutomationControlled")
        edge_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        edge_options.add_experimental_option('useAutomationExtension', False)
        
        driver = None
        
        try:
            service = Service(r"C:\Program Files (x86)\Microsoft\Edge\Application\141.0.3537.99\msedgedriver.exe")  # dùng đúng driver bản 140
            driver = webdriver.Edge(service=service, options=edge_options)
            
            # Ẩn dấu hiệu automation
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            
            # 2. Truy cập Bing trước
            print("🌐 Đang truy cập Bing.com...")
            driver.get("https://www.bing.com")
            
            # 3. Thử click nút sync nếu có
            if use_sync:
                sync_edge_account(driver)
            
            # 4. Tìm ô tìm kiếm và nhập từ khóa
            print(f"⌨️  Đang nhập từ khóa: '{keyword}'")
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "q"))
            )

            search_box.clear()
            search_box.send_keys(keyword)
            search_box.send_keys(Keys.RETURN)
            
            # 5. Chờ kết quả tìm kiếm hiển thị
            print("⏳ Đang chờ kết quả tìm kiếm...")
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "b_results"))
            )
            time.sleep(2)
            # 6. Tìm và click vào kết quả đầu tiên
            print("🎯 Đang tìm kết quả đầu tiên...")
            first_result = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#b_results .b_algo h2 a"))
            )
            
            first_result_title = first_result.text
            first_result_url = first_result.get_attribute("href")
            
            print(f"📄 Kết quả đầu tiên: {first_result_title}")
            print(f"🔗 URL: {first_result_url}")
            
            print("👆 Đang click vào kết quả đầu tiên...")
            first_result.click()
            
            # 7. Đợi 10 giây
            print("⏰ Đang đợi 10 giây...")
            time.sleep(6)
            
            print("✅ Hoàn thành!")
            
        except TimeoutException:
            print("❌ Lỗi: Timeout - Trang web load quá lâu")
        except NoSuchElementException:
            print("❌ Lỗi: Không tìm thấy element cần thiết")
        except Exception as e:
            print(f"❌ Lỗi không xác định: {e}")
        
        finally:
            # 8. Đóng browser
            if driver:
                print("🔴 Đang đóng browser...")
                kill_msedge_processes()                
                
        # Nghỉ 2 giây trước khi xử lý từ khóa tiếp theo
        if i < len(keywords):
            print("💤 Nghỉ 2 giây trước khi tiếp tục...")
            time.sleep(2)

# Cách sử dụng nhanh với đường dẫn cụ thể:
def quick_search_example():
    """
    Ví dụ sử dụng nhanh với đường dẫn profile cụ thể
    """
    # Thay đổi đường dẫn này theo máy của bạn
    custom_profile_path = r"C:\Users\Admin\AppData\Local\Microsoft\Edge\User Data"
    custom_profile_name = [
        # "Default", ## spiderman002v001
        "Profile 1", ## spiderman002v002,
        'Profile 3', ##  brycen.lazybz.test@gmail.com
        'Profile 1Profile 3', ## spiderman003v005@gmail.com
        
    ]
    keywords = [
          "Việt Nam",
        "Canada",
        "Úc",
        "Mỹ",
        "Nhật Bản",
        "Hàn Quốc",
        "Trung Quốc",
        "Thái Lan",
        "Singapore",
        "Malaysia",
        "Indonesia",
        "Ấn Độ",
        "Anh Quốc",
        "Pháp",
        "Đức",
        "Ý",
        "Tây Ban Nha",
        "Bồ Đào Nha",
        "Nga",
        "Brazil",
        "Argentina",
        "Mexico",
        "Ai Cập",
        "Nam Phi",
        "UAE",
        "văn hóa Mỹ",
        "ẩm thực Việt Nam",
        "du lịch Nhật Bản",
        "lễ hội Ấn Độ",
        "kiến trúc Pháp"
    ]
    
    for item in custom_profile_name:
        auto_bing_search(
        keywords=keywords,
        profile_path=custom_profile_path,
        profile_name=item,
        use_sync=False
    )
quick_search_example()

/n============================================================
🔍 [1/30] Đang xử lý từ khóa: 'Việt Nam'
🌐 Đang truy cập Bing.com...
⌨️  Đang nhập từ khóa: 'Việt Nam'
⏳ Đang chờ kết quả tìm kiếm...
🎯 Đang tìm kết quả đầu tiên...
📄 Kết quả đầu tiên: Việt Nam – Wikipedia tiếng Việt
🔗 URL: https://www.bing.com/ck/a?!&&p=64dfe9815f09ffeb92035453f3ebbf2e1681eca9b1fb5f19693edd65613fc533JmltdHM9MTc2MTg2ODgwMA&ptn=3&ver=2&hsh=4&fclid=0df68965-dcef-6828-0a4d-9f53dd8969ea&psq=Vi%e1%bb%87t+Nam&u=a1aHR0cHM6Ly92aS53aWtpcGVkaWEub3JnL3dpa2kvVmklRTElQkIlODd0X05hbQ&ntb=1
👆 Đang click vào kết quả đầu tiên...
⏰ Đang đợi 10 giây...
✅ Hoàn thành!
🔴 Đang đóng browser...
💤 Nghỉ 2 giây trước khi tiếp tục...
/n============================================================
🔍 [2/30] Đang xử lý từ khóa: 'Canada'
🌐 Đang truy cập Bing.com...
⌨️  Đang nhập từ khóa: 'Canada'
⏳ Đang chờ kết quả tìm kiếm...
🎯 Đang tìm kết quả đầu tiên...
📄 Kết quả đầu tiên: Canada - Wikipedia
🔗 URL: https://www.bing.com/ck/a?!&&p=2daaf65b6